Test running sixdb in Python3 interactively

In [ ]:
# In case there is a bug with auto completion of paths, temp bugfix according to
# https://github.com/ipython/ipykernel/issues/299
%config Completer.use_jedi=False 

# add location of SixDeskDB for Python 3+ to module search path
import sys
# (example) set path to your sixdeskdb folder
sys.path.append("/afs/cern.ch/work/m/mtitze/hl-lhc/git/SixDeskDB/") 

import sixdb

In [ ]:
# to update functions if they are modified externally
%load_ext autoreload
%autoreload 2

In [ ]:
# run sixdesk with a set of unequal emittances
emittances_x = [1.11, 1.53]
emittances_y = [3.48, 2.71, 2.42]

emittances_x = [3.57]
emittances_y = [2.87]

w = sixdb.SixDB(['', '-emit', emittances_x, emittances_y])

In [ ]:
# set a name for the sixdesk SQL database to be analyzed.
study_name = '/afs/cern.ch/work/m/mtitze/hl-lhc/testbed/test_study/lhc_b1_inj_2016_qp_14_ioct_20_emit_3.57_2.87.db'
#study_name = '/afs/cern.ch/work/m/mtitze/hl-lhc/testbed/lhc_b1_inj_2016_qp_0_ioct_40_emit_2.00_1.66_old.db'

In [ ]:
w.da_method = 1  # 0: slow loop of slightly modified original code, 1: fast loop. Method 0 currently generates seperate
# tables if there is more than one emittance pair.
w.da_vs_turns(study=study_name)

In [ ]:
# if necessary, drop tables
import itertools

import sqlite3

def drop_from_sql_database(filename, remove=['da_vst', 'fit_parameters'], verbose=True):
    '''
    This is a temporarily solution. We can remove tables from a sixdesk database
    in order to force/ensure that the values are re-computed if there is a change.
    '''
    if verbose:
        print ('Dropping {} from SQL database\n {}'.format(remove, filename))
    
    connection = sqlite3.connect(filename)
    cursor = connection.cursor()
    for table_name in remove:
        try:
            cursor.execute("DROP TABLE {}".format(table_name))
        except:
            # it might be that there is no such table etc.
            pass
    connection.close()

In [ ]:
to_remove = []
for ex, ey in itertools.product(emittances_x, emittances_y):
    to_remove.append('da_vst_{}_{}'.format(ex, ey).replace('.', ''))

#drop_from_sql_database(study_name, remove=to_remove)
drop_from_sql_database(study_name)